In [ ]:
import json
import requests

In [33]:
#Prosta serializacja
data = {
    "student": {
        "name": "Piotr Kowalecki",
        "group": "175IC_B1"
    }
}

#Zapisanie do pliku data_file o rozszerzeniu JSON
with open("json_file/data_file.json", "w") as write_file:
  json.dump(data, write_file)


json.dumps(data)

'{"student": {"name": "Piotr Kowalecki", "group": "175IC_B1"}}'

In [34]:
#Dodanie wycięć w wyświetlaniu danych
json.dumps(data, indent=3)

'{\n   "student": {\n      "name": "Piotr Kowalecki",\n      "group": "175IC_B1"\n   }\n}'

In [35]:
#Deserializacja z pliku
with open("json_file/data_file_deserialization.json", "r") as read_file:
    data = json.load(read_file)
data

{'student': {'group': '175IC_B1',
  'name': 'Piotr Kowalecki',
  'type': 'deserialization'}}

In [37]:
#Deserializacja z URL
response = requests.get("https://jsonplaceholder.typicode.com/todos")
todos = json.loads(response.text)
todos

[{'completed': False, 'id': 1, 'title': 'delectus aut autem', 'userId': 1},
 {'completed': False,
  'id': 2,
  'title': 'quis ut nam facilis et officia qui',
  'userId': 1},
 {'completed': False, 'id': 3, 'title': 'fugiat veniam minus', 'userId': 1},
 {'completed': True, 'id': 4, 'title': 'et porro tempora', 'userId': 1},
 {'completed': False,
  'id': 5,
  'title': 'laboriosam mollitia et enim quasi adipisci quia provident illum',
  'userId': 1},
 {'completed': False,
  'id': 6,
  'title': 'qui ullam ratione quibusdam voluptatem quia omnis',
  'userId': 1},
 {'completed': False,
  'id': 7,
  'title': 'illo expedita consequatur quia in',
  'userId': 1},
 {'completed': True,
  'id': 8,
  'title': 'quo adipisci enim quam ut ab',
  'userId': 1},
 {'completed': False,
  'id': 9,
  'title': 'molestiae perspiciatis ipsa',
  'userId': 1},
 {'completed': True,
  'id': 10,
  'title': 'illo est ratione doloremque quia maiores aut',
  'userId': 1},
 {'completed': True,
  'id': 11,
  'title': 'vero

In [38]:
#Wyświetlanie ukończonych zadań
todos_by_user = {}

for todo in todos:
  if todo["completed"]:
    try:
      todos_by_user[todo["userId"]] += 1
    except KeyError:
      todos_by_user[todo["userId"]] = 1

top_users = sorted(todos_by_user.items(), 
                   key=lambda x: x[1], reverse=True)

print(f'Top postów: {top_users}')

max_complete = top_users[0][1]
users = []
for user, num_complete in top_users:
    if num_complete < max_complete:
        break
    users.append(str(user))

max_users = " and ".join(users)

print(f'Osoby które ukończyły wszystkie taski: {max_users}')

Top postów: [(5, 12), (10, 12), (1, 11), (8, 11), (7, 9), (2, 8), (9, 8), (3, 7), (4, 6), (6, 6)]
Osoby które ukończyły wszystkie taski: 5 and 10


In [40]:
#Funkcja filtrująca ukończone zadania przez topowe ID 
def keep(todo):
    is_complete = todo["completed"]
    has_max_count = str(todo["userId"]) in users
    return is_complete and has_max_count

#Zapis danych do JSON
with open("json_file/filtered_data_file.json", "w") as data_file:
    filtered_todos = list(filter(keep, todos))
    json.dump(filtered_todos, data_file, indent=2)

filtered_todos

[{'completed': True, 'id': 81, 'title': 'suscipit qui totam', 'userId': 5},
 {'completed': True,
  'id': 83,
  'title': 'quidem at rerum quis ex aut sit quam',
  'userId': 5},
 {'completed': True, 'id': 85, 'title': 'et quia ad iste a', 'userId': 5},
 {'completed': True,
  'id': 86,
  'title': 'incidunt ut saepe autem',
  'userId': 5},
 {'completed': True,
  'id': 87,
  'title': 'laudantium quae eligendi consequatur quia et vero autem',
  'userId': 5},
 {'completed': True, 'id': 89, 'title': 'sequi ut omnis et', 'userId': 5},
 {'completed': True,
  'id': 90,
  'title': 'molestiae nisi accusantium tenetur dolorem et',
  'userId': 5},
 {'completed': True,
  'id': 91,
  'title': 'nulla quis consequatur saepe qui id expedita',
  'userId': 5},
 {'completed': True, 'id': 92, 'title': 'in omnis laboriosam', 'userId': 5},
 {'completed': True,
  'id': 93,
  'title': 'odio iure consequatur molestiae quibusdam necessitatibus quia sint',
  'userId': 5},
 {'completed': True,
  'id': 95,
  'title': 

In [45]:
#Kodowanie i dekodowanie niestandardowych obiektów Pythona
def encode_complex(z):
    if isinstance(z, complex):
        return (z.real, z.imag)
    else:
        type_name = z.__class__.__name__
        raise TypeError(f"Object of type '{type_name}' is not JSON serializable")
json.dumps(9 + 5j, default=encode_complex)

'[9.0, 5.0]'

In [56]:
#Kodowanie z pomocą JSONEncoder
class ComplexEncoder(json.JSONEncoder):
    def default(self, z):
        if isinstance(z, complex):
            return (z.real, z.imag)
        else:
            return super().default(z)

#Przykładowe wywołania kodowania powyżej
print(f'json.dumps: {json.dumps(2 + 6j, cls=ComplexEncoder)}')
encoder = ComplexEncoder()
print(f'encoder.encode: {encoder.encode(3 + 6j)}')

#Dekodowanie typów niestandardowych
complex_json = json.dumps(4 + 17j, cls=ComplexEncoder)
print(f'json.loads: {json.loads(complex_json)}')

json.dumps: [2.0, 6.0]
encoder.encode: [3.0, 6.0]
json.loads: [4.0, 17.0]


In [69]:
#Dekodowanie klucza - definicja oraz odkodowanie
def decode_complex(dct):
    if "__complex__" in dct:
        return complex(dct["real"], dct["imag"])
    return dct

with open("json_file/complex_data.json") as complex_data:
     data = complex_data.read()
     z = json.loads(data, object_hook=decode_complex)

print(z)

[(42+36j), (1+1j), (3+5j)]
